In [110]:
from __future__ import division
import pandas as pd
import uuid
import random
import numpy as np
import datetime, time
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
raw_df = pd.read_csv('/home/jasper/Downloads/user-actions000000000000', sep=',')

print raw_df.info()

raw_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354396 entries, 0 to 8354395
Data columns (total 3 columns):
date             int64
fullVisitorId    uint64
transactions     float64
dtypes: float64(1), int64(1), uint64(1)
memory usage: 191.2 MB
None


,date,fullVisitorId,transactions
0,20160930,1769651648414523235,NaN
1,20160930,1867790925252608084,NaN
2,20160930,2083829884801365723,NaN
3,20160930,272111073307289500,NaN
4,20160930,291837269665288281,NaN


For each day in the range, we need to calculate how many different visits there were.

For each visit we need:
    - Who the user was, if they were returning or not
    - Session start time
    - If they purchased
    - How many items they purchased
    - Which items they purchased (Product IDs)
    - The total transaction revenue
    - Google search terms
    - Channel
    - Onsite search terms
    - User Agent Stuff [
        device broser,
        browser version,
        OS,
        geo - town,
        lat,
        long]    

In [351]:
def how_many_visits_today(lower_bound, upper_bound):
    return np.random.randint(lower_bound, upper_bound)

def generate_session_id():
    return uuid.uuid4()

def generate_time_of_day():
    return {
        'hours': random.randrange(0, 23),
        'minutes': random.randrange(0, 59),
        'seconds': random.randrange(0, 59)
    }

def combine_times(input_date):
    return input_date + datetime.timedelta(**generate_time_of_day())
    
def was_product_purchased():
    X = random.random()
    if X >= .95:
        return True
    else:
        return False
    
def which_product_was_purchased():
    p_id = np.random.multinomial(1, [1/7.]*5 + [2/7.], size=1).argmax()
    return {'id': '{0}'.format(p_id)}

def how_much_was_the_product(p_id):
    X = {
        1: 10,
        2: 20,
        3: 30,
        4: 40,
        5: 50,
        0: 5}
    return X[p_id]

def which_marketing_channel():
    channels = {
        0: 'direct',
        1: 'seo',
        2: 'cpc',
        3: 'display',
        4: 'other'
    }
    return channels[np.random.multinomial(1, [1/5.]*5, size=1).argmax()]

def which_browser():
    browser = {
        0: 'edge',
        1: 'chrome',
        2: 'firefox',
        3: 'apple',
        4: 'other'
    }
    return browser[np.random.multinomial(1, [1/5.]*5, size=1).argmax()]


def which_OS():
    OS = {
        0: 'windows',
        1: 'linux',
        2: 'apple',
        3: 'gentoo'
    }
    return OS[np.random.multinomial(1, [5/10., 3/10., 3/20., 1/20.], size=1).argmax()]


def generate_user_record(date):
    return {
        'session_id': generate_session_id(),
        'visitStartTime': combine_times(d),
        'transaction': was_product_purchased(),
        'marketing_channel': which_marketing_channel(),
        'deviceBrowser': which_browser(),
        'which_OS': which_OS()}

In [346]:
user_list = []
for d in pd.date_range(start=start_date, end='2018-01-31'):
    user_list.append(generate_user_record(d))

In [352]:
df = pd.DataFrame(user_list)

print df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
deviceBrowser        31 non-null object
marketing_channel    31 non-null object
session_id           31 non-null object
transaction          31 non-null bool
visitStartTime       31 non-null datetime64[ns]
which_OS             31 non-null object
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 1.3+ KB
None


,deviceBrowser,marketing_channel,session_id,transaction,visitStartTime,which_OS
0,edge,seo,f2146512-682d-415f-8596-aef67a67df1e,False,2018-01-01 10:29:16,windows
1,edge,other,6b5a6548-08dd-442f-9dbc-394cd0fbb264,False,2018-01-02 21:12:39,windows
2,firefox,display,86eb4010-59cf-4d5e-9c41-11046494a8db,False,2018-01-03 22:55:39,windows
3,other,other,8da7060d-9968-43a1-8493-373fa4104a56,False,2018-01-04 11:54:50,windows
4,other,display,e4ada8e1-c02d-41ae-b637-46c411618fae,False,2018-01-05 19:16:42,windows


In [378]:
df.to_csv()

,deviceBrowser,marketing_channel,session_id,transaction,visitStartTime,which_OS
0,edge,seo,f2146512-682d-415f-8596-aef67a67df1e,False,2018-01-01 10:29:16,windows
1,edge,other,6b5a6548-08dd-442f-9dbc-394cd0fbb264,False,2018-01-02 21:12:39,windows
2,firefox,display,86eb4010-59cf-4d5e-9c41-11046494a8db,False,2018-01-03 22:55:39,windows
3,other,other,8da7060d-9968-43a1-8493-373fa4104a56,False,2018-01-04 11:54:50,windows
4,other,display,e4ada8e1-c02d-41ae-b637-46c411618fae,False,2018-01-05 19:16:42,windows
5,firefox,seo,90cd333f-2b45-44f9-9fba-0d740684e7f9,False,2018-01-06 20:56:05,linux
6,edge,other,bdd162f7-5801-4cd8-b278-b5a1d46023e9,False,2018-01-07 08:51:09,linux
7,firefox,other,82e12618-e34f-45b5-9aa3-b3bde42e91ca,False,2018-01-08 08:16:39,windows
8,chrome,seo,9892cf88-abe7-47b5-b4bc-52fe42cf8e53,False,2018-01-09 14:24:54,windows
9,apple,direct,2042c9f7-a160-4e2a-8463-ef90c431c770,False,2018-01-10 00:06:12,windows


In [195]:
which_product_was_purchased()

{'id': '5'}